# Import bibs

In [2]:
import pandas as pd

from utilities.Functionnalities import *
import warnings
warnings.filterwarnings("ignore")

# ENTRIES

In [3]:
# entries
dateObservation = 24
nbPool = 6
delta_hour = "8h"
n_i = "N"
star_time = "2021-07-21"
hour = "10:00:00"
ending_time = "2022-01-1"

## download node modules to use for collecting data with nodejs
### npm install ccxt
### npm install csv-writer

In [3]:
install_package_if_needed("ccxt")

ccxt is already installed.


In [4]:
install_package_if_needed("csv-writer")

csv-writer is already installed.


# La partie Itérative

In [4]:
data = {'current_date': pd.to_datetime(['2020-01-02 10:00:00']),
        'pool': [['BNB', 'ATD', 'ACC']]}
df = pd.DataFrame(data)
df.set_index('current_date', inplace=True)
df.drop(df.index, inplace=True)

command = 'node ./database/dl_for_quick_analysis.js'
path = "./database/quick_analysis"
ennDate = f"{ending_time} {hour}"
sttDate = f"{star_time} {hour}"
start_date = datetime.strptime(sttDate, "%Y-%m-%d %H:%M:%S")
end_date = datetime.strptime(ennDate, "%Y-%m-%d %H:%M:%S")

current_date = start_date

### Juste here lazem na3ref kifech fi kol nhar nzid nsob men 8ir manefsa5

In [11]:
def downloadingDate(current_date):
    execute_terminal_command2(command, current_date.strftime("%Y-%m-%d"), hour)
    #remove unecessary files just keep the csv files
    remove_non_csv_files(path)
    #Process Started

In [12]:
threads = []
while current_date <= end_date:
    thread = threading.Thread(target=downloadingDate, args=(current_date,))
    thread.start()
    threads.append(thread)
    current_date += timedelta(days=1)

# Wait for all threads to finish
for thread in threads:
    thread.join()
print("Process is done!")


node ./database/dl_for_quick_analysis.js "2023-05-01" "00:00:00"
node ./database/dl_for_quick_analysis.js "2023-05-02" "00:00:00"
node ./database/dl_for_quick_analysis.js "2023-05-03" "00:00:00"
node ./database/dl_for_quick_analysis.js "2023-05-04" "00:00:00"
node ./database/dl_for_quick_analysis.js "2023-05-05" "00:00:00"
node ./database/dl_for_quick_analysis.js "2023-05-06" "00:00:00"
node ./database/dl_for_quick_analysis.js "2023-05-07" "00:00:00"
node ./database/dl_for_quick_analysis.js "2023-05-08" "00:00:00"
node ./database/dl_for_quick_analysis.js "2023-05-09" "00:00:00"
node ./database/dl_for_quick_analysis.js "2023-05-10" "00:00:00"
Donwloading Finished
Command execution failed with return code: 1
Donwloading Finished
Command execution failed with return code: 1
Donwloading Finished
Command execution failed with return code: 1
Donwloading Finished
Command execution failed with return code: 1
Donwloading Finished
Command execution failed with return code: 1
Donwloading Finished

## get the data from the database/quick_analysis

In [ ]:
file_list = [f for f in listdir(path) if isfile(join(path, f))]
df_list = {}
for file in file_list:
    df_list[file[:-9]] = get_historical_from_path(path + "/" + file)
print("All data loaded")

df_metric = get_analyisis_from_window(df_list,dateObservation).sort_values(by="volume_evolution", ascending=False)
dfVe = df_metric.iloc[:nbPool]
market = list(dfVe.index)
df.loc[current_date.strftime("%Y-%m-%d %H:%M:%S")] = [list(dfVe['volume_evolution'].index)]
df_metric = get_analyisis_from_window(df_list,dateObservation).sort_values(by="volume_evolution", ascending=False)
dfVe = df_metric.iloc[:nbPool]
market = list(dfVe.index)
df.loc[current_date.strftime("%Y-%m-%d %H:%M:%S")] = [list(dfVe['volume_evolution'].index)]

In [13]:
df.to_csv('./database/pools.csv',header=False)
df

,pool
current_date,
2023-05-01 00:00:00,"[ALPHA-USDT, ALPHA, AGLD, ARDR, AKRO, ADA-USDT]"
2023-05-09 00:00:00,"[ALPHA-USDT, ALPHA, AGLD, ARDR, AKRO, ADA-USDT]"
2023-05-05 00:00:00,"[ALPHA-USDT, ALPHA, AGLD, ARDR, AKRO, ADA-USDT]"
2023-05-03 00:00:00,"[ALPHA-USDT, ALPHA, AGLD, ARDR, AKRO, ADA-USDT]"
2023-05-06 00:00:00,"[ALPHA-USDT, ALPHA, AGLD, ARDR, AKRO, ADA-USDT]"
2023-05-02 00:00:00,"[ALPHA-USDT, ALPHA, AGLD, ARDR, AKRO, ADA-USDT]"
2023-05-10 00:00:00,"[ALPHA-USDT, ALPHA, AGLD, ARDR, AKRO, ADA-USDT]"
2023-05-07 00:00:00,"[ALPHA-USDT, ALPHA, AGLD, ARDR, AKRO, ADA-USDT]"
2023-05-08 00:00:00,"[ALPHA-USDT, ALPHA, AGLD, ARDR, AKRO, ADA-USDT]"


# The BackTest With diffirent Combination

##  Generate all combinations


In [5]:
client = Client()
crypto = {}
deltaHours = ["2h","4h","8h","12h"]
Ni = ["N","N-1","N-2"]
array1 = []
with open('database/pools.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        timestamp = row[0]
        elements = row[1].strip('[]').split(', ')
        array1.append((timestamp, elements))
combinations = list(itertools.product(array1, deltaHours, Ni))
combinations = [list(item) for item in combinations]

In [7]:
for i in array1 :
    print(i)

('2023-05-01 00:00:00', ["'ALPHA-USDT'", "'ALPHA'", "'AGLD'", "'ARDR'", "'AKRO'", "'ADA-USDT'"])
('2023-05-09 00:00:00', ["'ALPHA-USDT'", "'ALPHA'", "'AGLD'", "'ARDR'", "'AKRO'", "'ADA-USDT'"])
('2023-05-05 00:00:00', ["'ALPHA-USDT'", "'ALPHA'", "'AGLD'", "'ARDR'", "'AKRO'", "'ADA-USDT'"])
('2023-05-03 00:00:00', ["'ALPHA-USDT'", "'ALPHA'", "'AGLD'", "'ARDR'", "'AKRO'", "'ADA-USDT'"])
('2023-05-06 00:00:00', ["'ALPHA-USDT'", "'ALPHA'", "'AGLD'", "'ARDR'", "'AKRO'", "'ADA-USDT'"])
('2023-05-02 00:00:00', ["'ALPHA-USDT'", "'ALPHA'", "'AGLD'", "'ARDR'", "'AKRO'", "'ADA-USDT'"])
('2023-05-10 00:00:00', ["'ALPHA-USDT'", "'ALPHA'", "'AGLD'", "'ARDR'", "'AKRO'", "'ADA-USDT'"])
('2023-05-07 00:00:00', ["'ALPHA-USDT'", "'ALPHA'", "'AGLD'", "'ARDR'", "'AKRO'", "'ADA-USDT'"])
('2023-05-08 00:00:00', ["'ALPHA-USDT'", "'ALPHA'", "'AGLD'", "'ARDR'", "'AKRO'", "'ADA-USDT'"])
('2023-05-04 00:00:00', ["'ALPHA-USDT'", "'ALPHA'", "'AGLD'", "'ARDR'", "'AKRO'", "'ADA-USDT'"])


## Prepare the Cocotier Method

In [15]:
import traceback
def cocotier(combination):
    crypto = {}
    x = ""
    for elm in combination[0][1] :
        try :
            x = elm.replace("'","")
            #get the values from binance as they are and convert them to a dataframe we can work with
            if x.endswith("-USDT"):
                x = x[:-5]
            else:
                x = x
            x = x.lower()+'/usdt'
            x = ''.join(char for char in x if not char.isdigit())
            ennDate = combination[0][0]
            previousDate = (datetime.strptime(ennDate, "%Y-%m-%d %H:%M:%S") - timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
            crypto[x] = client.get_historical_klines(x.replace("/", "").upper(), combination[1], previousDate,ennDate)
            crypto[x] = pd.DataFrame(data=crypto[x],
                                     columns=['timestamp', x[:-5] + '_open', 'high', 'low', x[:-5] + '_close', 'volume','close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore'])
            crypto[x] = crypto[x][['timestamp', x[:-5] + '_open', x[:-5] + '_close']]
            crypto[x] = convert_time(crypto[x])
            crypto[x] = crypto[x].astype({x[:-5] + '_open': 'float64',x[:-5] + '_close': 'float64'})
            crypto[x] = crypto[x].set_index('timestamp')
        except Exception as ll:
            print(f"{ll}\n{x} :  Please Try another name of this crypto !")
            traceback.format_exc()
    try :
        array_mauvais_shape = detection_mauvais_shape(crypto)
        # crypto = correction_shape(crypto, array_mauvais_shape)
        # for elm in array_mauvais_shape:
        #     crypto[elm]['timestamp'] = generation_date(crypto[elm], int(delta_hour[:1]))
        #     crypto[elm] = crypto[elm].set_index('timestamp')
        for i in array_mauvais_shape:
            del crypto[i]
        crypto = variationN(crypto, combination[2])
        crypto = coeffMulti(crypto)
        crypto = mergeCryptoTogether(crypto)
        crypto, maxis = botMax(crypto)
        crypto = botMaxVariation2(crypto, maxis)
        crypto = coeffMultiBotMax(crypto)
        coefMulti = coefmultiFinal(crypto)
        combination.append(coefMulti.tail(1).iloc[-1,-1])
    except:
        traceback.format_exc()

In [16]:
# Launch a thread for each iteration
threads = []
combinations = list(combinations)
for combination in combinations:
    thread = threading.Thread(target=cocotier, args=(combination,))
    thread.start()
    threads.append(thread)

# Wait for all threads to finish
for thread in threads:
    thread.join()
# cocotier(combination)

In [17]:

# Flatten the nested lists
flat_data = []
for combination in combinations:
    datetime = combination[0][0]
    pool = combination[0][1]
    deltahour = combination[1]
    Ni = combination[2]
    BotMax = combination[3]
    flat_data.append([datetime, deltahour, Ni, BotMax, pool])
df = pd.DataFrame(flat_data, columns=['datetime', 'deltahour', 'Ni', 'BotMax','pool'])


In [18]:
df

,datetime,deltahour,Ni,BotMax,pool
0,2023-05-01 00:00:00,2h,N,0.886147,"['ALPHA-USDT', 'ALPHA', 'AGLD', 'ARDR', 'AKRO'..."
1,2023-05-01 00:00:00,2h,N-1,0.886147,"['ALPHA-USDT', 'ALPHA', 'AGLD', 'ARDR', 'AKRO'..."
2,2023-05-01 00:00:00,2h,N-2,0.886147,"['ALPHA-USDT', 'ALPHA', 'AGLD', 'ARDR', 'AKRO'..."
3,2023-05-01 00:00:00,4h,N,0.890107,"['ALPHA-USDT', 'ALPHA', 'AGLD', 'ARDR', 'AKRO'..."
4,2023-05-01 00:00:00,4h,N-1,0.890107,"['ALPHA-USDT', 'ALPHA', 'AGLD', 'ARDR', 'AKRO'..."
...,...,...,...,...,...
115,2023-05-04 00:00:00,8h,N-1,1.001637,"['ALPHA-USDT', 'ALPHA', 'AGLD', 'ARDR', 'AKRO'..."
116,2023-05-04 00:00:00,8h,N-2,1.001637,"['ALPHA-USDT', 'ALPHA', 'AGLD', 'ARDR', 'AKRO'..."
117,2023-05-04 00:00:00,12h,N,1.006352,"['ALPHA-USDT', 'ALPHA', 'AGLD', 'ARDR', 'AKRO'..."
118,2023-05-04 00:00:00,12h,N-1,1.006352,"['ALPHA-USDT', 'ALPHA', 'AGLD', 'ARDR', 'AKRO'..."


In [19]:
# Convert 'datetime' column to datetime format
df['datetime'] = pd.to_datetime(df['datetime'])

# Group DataFrame by 'deltahour' and 'Ni', calculate product of 'BotMax'
df_grouped = df.groupby(['deltahour', 'Ni']).agg({'datetime': ['min', 'max'], 'BotMax': 'prod'}).reset_index()

# Rename columns
df_grouped.columns = ['deltahour', 'Ni', 'startDate', 'endingDate', 'BotMax']

# Convert 'startDate' and 'endingDate' columns to desired format
df_grouped['startDate'] = df_grouped['startDate'].dt.strftime('%Y-%m-%d %H:%M:%S')
df_grouped['endingDate'] = df_grouped['endingDate'].dt.strftime('%Y-%m-%d %H:%M:%S')


In [20]:
df_grouped

,deltahour,Ni,startDate,endingDate,BotMax
0,12h,N,2023-05-01 00:00:00,2023-05-10 00:00:00,0.573131
1,12h,N-1,2023-05-01 00:00:00,2023-05-10 00:00:00,0.573131
2,12h,N-2,2023-05-01 00:00:00,2023-05-10 00:00:00,0.573131
3,2h,N,2023-05-01 00:00:00,2023-05-10 00:00:00,0.607870
4,2h,N-1,2023-05-01 00:00:00,2023-05-10 00:00:00,0.607870
5,2h,N-2,2023-05-01 00:00:00,2023-05-10 00:00:00,0.607870
6,4h,N,2023-05-01 00:00:00,2023-05-10 00:00:00,0.580130
7,4h,N-1,2023-05-01 00:00:00,2023-05-10 00:00:00,0.580130
8,4h,N-2,2023-05-01 00:00:00,2023-05-10 00:00:00,0.580130
9,8h,N,2023-05-01 00:00:00,2023-05-10 00:00:00,0.552806


# Bug de téléchargement des données.

In [5]:
import pandas as pd

from utilities.Functionnalities import *
import warnings
warnings.filterwarnings("ignore")

# entries
dateObservation = 24
nbPool = 6
delta_hour = "8h"
n_i = "N"
star_time = "2023-01-01"
hour = "10:00:00"
ending_time = "2023-01-02"


data = {'current_date': pd.to_datetime(['2020-01-02 10:00:00']),
        'pool': [['BNB', 'ATD', 'ACC']]}
df = pd.DataFrame(data)
df.set_index('current_date', inplace=True)
df.drop(df.index, inplace=True)

command = 'node ./database/dl_for_quick_analysis.js'
path = "./database/quick_analysis"
ennDate = f"{ending_time} {hour}"
sttDate = f"{star_time} {hour}"
start_date = datetime.strptime(sttDate, "%Y-%m-%d %H:%M:%S")
end_date = datetime.strptime(ennDate, "%Y-%m-%d %H:%M:%S")

current_date = start_date

execute_terminal_command2(command, current_date.strftime("%Y-%m-%d"), hour)
#remove unecessary files just keep the csv files
remove_non_csv_files(path)
#Process Started
file_list = [f for f in listdir(path) if isfile(join(path, f))]
df_list = {}
for file in file_list:
    df_list[file[:-9]] = get_historical_from_path(path + "/" + file)
print("All data loaded")

df_metric = get_analyisis_from_window(df_list,dateObservation).sort_values(by="volume_evolution", ascending=False)
dfVe = df_metric.iloc[:nbPool]

node ./database/dl_for_quick_analysis.js "2023-01-01" "10:00:00"
Donwloading Finished
Command executed successfully.
All data loaded


In [5]:
print(dfVe['volume_evolution'])
print(current_date)
#star_time = "2023-01-01"

RAD-USDT     13.85
SYS          10.08
SKL-USDT      9.86
DUSK-USDT     9.31
CELR-USDT     7.92
ZEN           7.29
Name: volume_evolution, dtype: float64


In [7]:
print(dfVe['volume_evolution'])
print(current_date)
#star_time = "2023-01-02"

SXP-USDT     15.86
RAD-USDT     13.85
FIRO         13.16
SXP          11.50
KEY           9.61
DUSK-USDT     9.16
Name: volume_evolution, dtype: float64


In [9]:
print(dfVe['volume_evolution'])
print(current_date)
#star_time = "2023-01-01"

AUTO        2.48
AMB-USDT    1.31
ACH         1.16
AGLD        1.07
ACM         0.79
AR-USDT     0.77
Name: volume_evolution, dtype: float64


In [11]:
print(dfVe['volume_evolution'])
print(current_date)
#star_time = "2023-01-01"

AUTO         3.86
AGLD         3.28
BEAM         2.32
ARPA-USDT    2.17
ATA-USDT     1.64
AR-USDT      1.40
Name: volume_evolution, dtype: float64


In [2]:
print(dfVe['volume_evolution'])
print(current_date)
#star_time = "2023-01-01"

RAD-USDT     13.62
SYS          10.08
SKL-USDT      9.86
DUSK-USDT     9.31
CELR-USDT     7.92
ZEN           7.29
Name: volume_evolution, dtype: float64


In [4]:
print(dfVe['volume_evolution'])
print(current_date)
#star_time = "2023-01-01"

AUTO         3.86
AGLD         3.28
BEAM         2.32
ARPA-USDT    2.17
ATA-USDT     1.64
ATA          1.40
Name: volume_evolution, dtype: float64


In [7]:
print(dfVe['volume_evolution'])
print(current_date)
#star_time = "2023-01-01"

RAD-USDT     13.62
SYS          10.08
SKL-USDT      9.86
DUSK-USDT     9.31
CELR-USDT     7.92
ZEN           7.29
Name: volume_evolution, dtype: float64
2023-01-01 10:00:00
